In [40]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torchvision import models

from torchsummary import summary # pip install torchsummary

# Trial1 (Encoder ResNet)
- [Code Link](https://github.com/pytorch/vision/blob/v0.2.0/torchvision/models/resnet.py)
- [Paper Link](https://arxiv.org/abs/1512.03385)
    - Refer Table1 for ResNet-18/34/50

In [2]:

def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1
    
    def __init__(self, in_channels, channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        
        self.conv1 = conv3x3(in_channels, channels, stride)
        self.bn1   = nn.BatchNorm2d(channels)
        self.relu  = nn.ReLU(inplace=True)
        
        self.conv2 = conv3x3(channels, channels)
        self.bn2   = nn.BatchNorm2d(channels)
        
        self.downsample = downsample
        self.stride     = stride
    
    def forward(self, x):
        identity = x
        out      = self.conv1(x)
        out      = self.bn1(out)
        out      = self.relu(out)
        
        out      = self.conv2(out)
        out      = self.bn2(out)
        
        if self.downsample is not None:
            identity = self.downsample(x)
            
        out  += identity
        out = self.relu(out)
        
        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1   = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2   = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3   = nn.BatchNorm2d(planes * self.expansion)
        self.relu  = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

    

class ResNet(nn.Module):
    
    def __init__(self, block, layers, num_classes=10, zero_init_residual=False):
        super(ResNet, self).__init__()

        self.in_channels = 64
        self.conv1 = nn.Conv2d(3,64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1   = nn.BatchNorm2d(64)
        self.relu  = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1])
        self.layer3 = self._make_layer(block, 256, layers[2])
        self.layer4 = self._make_layer(block, 512, layers[3])

        self.avgpool = nn.AdaptiveAvgPool2d((1,1))
        self.fc      = nn.Linear(512*block.expansion, num_classes)
        
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias,0)
        
        if zero_init_residual:
            for m in self.modules():
                if isinstance(m.bin3.weight, 0):
                    nn.init.constant_(m.nbn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight,0)

    def _make_layer(self, block, channels, blocks, stride=1):
        downsample = None
        
        # print ('Stride : ', stride, ' || self.in_channels :', self.in_channels, ' || Channels : ',channels, block.expansion, channels * block.expansion)
        # print (stride != 1, self.in_channels != channels * block.expansion)
        if stride != 1 or self.in_channels != channels * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.in_channels, channels * block.expansion, stride),
                nn.BatchNorm2d(channels * block.expansion)                
            )
        
        layers = []
        layers.append(block(self.in_channels, channels, stride, downsample))
        self.in_channels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, channels))

        return nn.Sequential(*layers)
        
    
    def forward(self ,x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x 

def resnet18():
    model = ResNet(BasicBlock, [2, 2, 2, 2])
    return model

def resnet34():
    model = ResNet(BasicBlock, [3, 4, 6, 3])
    return model

In [8]:
if __name__ == "__main__":
     
    net18 = resnet18().cuda()  # len(list(net18.children())) = 10
    # net18 = ResNet(Bottleneck, [2, 2, 2, 2])  # len(list(net18.children())) = 10
    summary(net18, input_size=(3, 512, 512))    
    # print (len(list(net18.children())))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 256, 256]           9,408
       BatchNorm2d-2         [-1, 64, 256, 256]             128
              ReLU-3         [-1, 64, 256, 256]               0
         MaxPool2d-4         [-1, 64, 128, 128]               0
            Conv2d-5         [-1, 64, 128, 128]          36,864
       BatchNorm2d-6         [-1, 64, 128, 128]             128
              ReLU-7         [-1, 64, 128, 128]               0
            Conv2d-8         [-1, 64, 128, 128]          36,864
       BatchNorm2d-9         [-1, 64, 128, 128]             128
             ReLU-10         [-1, 64, 128, 128]               0
       BasicBlock-11         [-1, 64, 128, 128]               0
           Conv2d-12         [-1, 64, 128, 128]          36,864
      BatchNorm2d-13         [-1, 64, 128, 128]             128
             ReLU-14         [-1, 64, 1

# Trial2 (Encoder-Decoder ResNet)
- [Link](https://github.com/usuyama/pytorch-unet/blob/master/pytorch_resnet18_unet.ipynb)

In [65]:
def convrelu(in_channels, out_channels, kernel, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel, padding=padding),
        nn.ReLU(inplace=True)
    )

class ResNetUnet(nn.Module):
    
    def __init__(self, n_classes, verbose=0):
        super().__init__()
        
        self.verbose     = verbose 
        
        self.base_model  = models.resnet18(pretrained=False)
        self.base_layers = list(self.base_model.children())
        
        self.layer0      = nn.Sequential(*self.base_layers[:3])
        self.layer0_1x1  = convrelu(64,64,1,0)
        
        self.layer1      = nn.Sequential(*self.base_layers[3:5])
        self.layer1_1x1  = convrelu(64,64,1,0)
        
        self.layer2      = self.base_layers[5]
        # self.layer2_1x1  = convrelu(512, 512, 1,0)
        self.layer2_1x1  = convrelu(128, 64, 1,0)
        
        self.layer3      = self.base_layers[6]
        # self.layer3_1x1  = convrelu(1024,512,1,0)
        self.layer3_1x1  = convrelu(256,128,1,0)
        
        self.layer4      = self.base_layers[7]
        # self.layer4_1x1  = convrelu(2048, 1024,1,0)
        self.layer4_1x1  = convrelu(512, 256,1,0)
        
        self.upsample    = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        
        # self.conv_up3    = convrelu(512+1024 , 512, 3, 1)
        self.conv_up3    = convrelu(128+256 , 128, 3, 1)
        #self.conv_up2    = convrelu(512 + 512, 512, 3, 1) 128+64
        self.conv_up2    = convrelu(128+64, 64, 3, 1)
        #self.conv_up1    = convrelu(256 + 512, 256, 3, 1)
        self.conv_up1    = convrelu(64 + 64, 64, 3, 1)
        # self.conv_up0    = convrelu(64 + 256 , 128, 3, 1)
        self.conv_up0    = convrelu(64 + 64 , 64, 3, 1)
        
        self.conv_original_size0 = convrelu(3, 32, 3, 1)
        self.conv_original_size1 = convrelu(32, 64, 3, 1)
        self.conv_original_size2 = convrelu(128, 64, 3, 1)
        
        self.conv_last   = nn.Conv2d(64, n_classes, 1)
    
    def forward(self, x_input):
        if self.verbose: print ('Preprocess : x_input    : ', x_input.shape)
        x_original = self.conv_original_size0(x_input)
        if self.verbose: print ('Preprocess : x_original : ', x_original.shape)
        x_original = self.conv_original_size1(x_original)
        if self.verbose: print ('Preprocess : x_original : ', x_original.shape)
        
        
        
        layer0 = self.layer0(x_input)
        layer1 = self.layer1(layer0)
        layer2 = self.layer2(layer1)
        layer3 = self.layer3(layer2)
        layer4 = self.layer4(layer3)
        
        if self.verbose:
            print ('\nEncoder : Layer0 : ', layer0.shape)
            print ('Encoder : Layer1 : ', layer1.shape)
            print ('Encoder : Layer2 : ', layer2.shape)
            print ('Encoder : Layer3 : ', layer3.shape)
            print ('Encoder : Layer4 : ', layer4.shape)
        
        layer4 = self.layer4_1x1(layer4)
        if self.verbose: print ('\nDecoder : Layer4 : ', layer4.shape)
        
        x      = self.upsample(layer4)
        layer3 = self.layer3_1x1(layer3)
        x      = torch.cat([x, layer3], dim=1)
        x      = self.conv_up3(x)
        if self.verbose: print ('Decoder : Layer3 : ', x.shape)
                
        x      = self.upsample(x)
        layer2 = self.layer2_1x1(layer2)
        x      = torch.cat([x, layer2], dim=1)
        x      = self.conv_up2(x)
        if self.verbose: print ('Decoder : Layer2 : ', x.shape)
        
        x      = self.upsample(x)
        layer1 = self.layer1_1x1(layer1)
        x      = torch.cat([x, layer1], dim=1)
        x      = self.conv_up1(x)
        if self.verbose: print ('Decoder : Layer1 : ', x.shape) # [1, 64, h/4, w/4]
        
        x      = self.upsample(x)
        layer0 = self.layer0_1x1(layer0)
        x      = torch.cat([x, layer0], dim=1)
        x      = self.conv_up0(x)
        if self.verbose:print ('Decoder : Layer0 : ', x.shape) # [1, 64, h/2, w/2]
        
        x      = self.upsample(x)
        x      = torch.cat([x, x_original], dim=1)
        x      = self.conv_original_size2(x)
        if self.verbose : print ('Decoder : x    : ', x.shape) 
        
        out    = self.conv_last(x)
        if self.verbose : print ('Decoder : out  : ', out.shape)
        
        return F.sigmoid(out)
        

In [67]:
net = ResNetUnet(1).cuda()
summary(net, input_size=(3,512, 512)) 


# net =  ResNetUnet(1, verbose=1)
# x  = torch.randn(1,3,512,512)
# y = net(x)
# y.shape
# layers = list(net.children())
# print (len(layers))
 

/home/strider/anaconda3/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 512, 512]             896
              ReLU-2         [-1, 32, 512, 512]               0
            Conv2d-3         [-1, 64, 512, 512]          18,496
              ReLU-4         [-1, 64, 512, 512]               0
            Conv2d-5         [-1, 64, 256, 256]           9,408
            Conv2d-6         [-1, 64, 256, 256]           9,408
       BatchNorm2d-7         [-1, 64, 256, 256]             128
       BatchNorm2d-8         [-1, 64, 256, 256]             128
              ReLU-9         [-1, 64, 256, 256]               0
             ReLU-10         [-1, 64, 256, 256]               0
        MaxPool2d-11         [-1, 64, 128, 128]               0
        MaxPool2d-12         [-1, 64, 128, 128]               0
           Conv2d-13         [-1, 64, 128, 128]          36,864
           Conv2d-14         [-1, 64, 1

/home/strider/anaconda3/lib/python3.5/site-packages/torch/nn/functional.py:1332: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [17]:
import torchvision
torchvision.__version__

'0.2.1'

# Visualization (Ignore)

In [ ]:
import torch
from torch import nn
from torchviz import make_dot, make_dot_from_trace

In [ ]:
model = nn.Sequential()
model.add_module('W0', nn.Linear(8, 16))
model.add_module('tanh', nn.Tanh())
model.add_module('W1', nn.Linear(16, 1))

x = torch.randn(1,8)

make_dot(model(x), params=dict(model.named_parameters()))